# Naver Movie Review Crawling

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re
import math

In [ ]:
def get_review_info(movie_nm):
    review_url, num_pages = None, 0
    search_url = "https://movie.naver.com/movie/search/result.nhn?query=" + movie_nm + "&section=all&ie=utf8"
    search_resp = requests.get(search_url)
    search_html = BeautifulSoup(search_resp.content, 'html.parser')
    if search_html.find('ul', {'class': 'search_list_1'}) is not None: # 해당 영화 검색 결과가 존재하는 경우
        a_tag = search_html.find('ul', {'class': 'search_list_1'}).find('a')
        re_movie = re.compile('code=[0-9]{1,6}')
        movie_code = re.sub('code=', '', re_movie.findall(str(a_tag))[0])
        review_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + movie_code
        review_html = BeautifulSoup(requests.get(review_url).content, 'html.parser')
        review_score = review_html.find('div', {'class': 'score_total'})
        if review_score is not None: # 해당 영화 리뷰 페이지가 존재하는 경우 (국내개봉작)
            review_count = int(review_score.find('strong').findChildren('em')[-1].getText().replace(',', ''))
            num_pages = int(math.ceil(review_count/10))
    return(review_url, num_pages)

In [ ]:
def get_movie_review_subset(movie_nm, url):
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    lis_df = pd.DataFrame()
    for li in lis:
        nickname = li.findAll('a')[0].find('span').getText() if li.findAll('a')[0].find('span') is not None else None
        created_at = datetime.strptime(li.find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
        review_text = li.find('p').getText().translate(str.maketrans({"\n": "", "\r": "", "\t": ""}))
        score = li.find('em').getText()
        btn_likes = li.find('div', {'class': 'btn_area'}).findAll('strong')
        like = btn_likes[0].getText()
        dislike = btn_likes[1].getText()
        watch_movie = li.find('span', {'class':'ico_viewer'})
        
        li_df = pd.DataFrame({"movie_nm": [movie_nm],
                              "nickname": [nickname],
                              "review": [review_text],
                              "score": [score],
                              "like": [like],
                              "dislike": [dislike],
                              "created at": [created_at],
                              "watch_movie": [watch_movie and True or False]})
        lis_df = pd.concat([lis_df, li_df])
    return(lis_df)

In [145]:
def temp_get_score(movie_nm):
    review_url, num_pages = None, 0
    search_url = "https://movie.naver.com/movie/search/result.nhn?query=" + movie_nm + "&section=all&ie=utf8"
    search_resp = requests.get(search_url)
    search_html = BeautifulSoup(search_resp.content, 'html.parser')
    
    watcher_score, expert_score, netizen_score = '', '', ''
    if search_html.find('ul', {'class': 'search_list_1'}) is not None: # 해당 영화 검색 결과가 존재하는 경우
        a_tag = search_html.find('ul', {'class': 'search_list_1'}).find('a')
        re_movie = re.compile('code=[0-9]{1,6}')
        movie_code = re.sub('code=', '', re_movie.findall(str(a_tag))[0])
        #################################################################
        score_url = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=" + movie_code
        resp = requests.get(score_url)
        html = BeautifulSoup(resp.content, 'html.parser')
        main_score = html.find('div', {'class': 'main_score'})
        
        if main_score is not None:
            score_tag = main_score.findAll('div', {'class': 'star_score'})[0].findAll('em')
            watcher_score = re.sub('>|<', '', "".join([str(i) for i in re.compile('>[0-9, .]<').findall("".join([str(i) for i in score_tag]))]))
            score_tag = main_score.findAll('div', {'class': 'star_score'})[1].findAll('em')
            expert_score = re.sub('>|<', '', "".join([str(i) for i in re.compile('>[0-9, .]<').findall("".join([str(i) for i in score_tag]))]))
            score_tag = main_score.findAll('div', {'class': 'star_score'})[2].findAll('em')
            netizen_score = re.sub('>|<', '', "".join([str(i) for i in re.compile('>[0-9, .]<').findall("".join([str(i) for i in score_tag]))]))
    return(watcher_score, expert_score, netizen_score)

In [ ]:
mv_std_info = pd.read_table("~/MovieTrends/data/mv_std_info.dat", sep = "|", encoding = "UTF-8")
mv_std_info.open_date = pd.Series([pd.Timestamp(datetime.strptime(i, '%Y%m%d')) for i in map(str, mv_std_info.open_date)])
mv_std_info = mv_std_info[["mv_id", "title", "open_date"]]

In [ ]:
movie_reviews_list = []
piece_size = 100
for piece_idx in range(0, len(mv_std_info), piece_size):
    movie_list = mv_std_info["title"][piece_idx:piece_idx+piece_size]
    movie_reviews = pd.DataFrame()
    for movie_nm in movie_list:
        review_url, num_pages = get_review_info(movie_nm)
        print(list(mv_std_info["title"]).index(movie_nm), "|", movie_nm, "|", num_pages, "pages |", review_url)
        movie_review = pd.DataFrame()
#         for i in range(1, num_pages+1): # 각 영화 모든 리뷰 수집
        for i in range(1, min(num_pages+1, 2)): # 각 영화 리뷰 최대 100개씩 수집
#             print(movie_nm + " | " + str(i) + "/" + str(num_pages))
            movie_review_subset = get_movie_review_subset(movie_nm, review_url + '&page=' + str(i))
            movie_review = pd.concat([movie_review, movie_review_subset])
        movie_reviews = pd.concat([movie_reviews, movie_review])
    movie_reviews_list.append(movie_reviews.reset_index().drop(['index'], axis=1))

In [146]:
scores = pd.DataFrame()
for mv in mv_std_info.title:
    print(mv)
    score = temp_get_score(mv)
    score_row = pd.DataFrame({'title': mv,
                              'watcher_score': [score[0]],
                              'expert_score': [score[1]],
                              'netizen_score': [score[2]]})
    scores = pd.concat([scores, score_row], axis=0)

언니
데드풀 2: 순한맛
레인보우: 나의 사랑
레토
미션 이스탄불 5: 더 파이널
주먹왕 랄프 2: 인터넷 속으로
아이스브레이커
그린 북
내안의 그놈
말모이
극장판 공룡메카드: 타이니소어의 섬
돈 겟 아웃!
디스 크레이지 하트
리지
빌리어네어 보이즈클럽
사무라이-검신 주베에
신 배틀로얄:극악 소년원
신 배틀로얄:극악 소년원 2
완벽한 그녀의 비밀
요코하마 갱스터
아스트로: 우주전쟁
구스 베이비
그대 이름은 장미
미래의 미라이
언더독
왕이 될 아이
쿠르스크
글래스
더 서치
디 엔드?
메이트
언싱커블
우행록: 어리석은 자의 기록
일일시호일
창간호
극한직업
두 번째 여름, 두 번 다시 만날 수 없는 너
몬스터파크
크리미널 게임: 보석 사기단
행맨
가버나움
데들리 스토커
베스와 베라
아이 킬 자이언츠
얼굴들
파이널리스트
하트스트링스
트라이아스기 월드: 잃어버린 세계
고검기담: 소명신검의 부활
드래곤 길들이기 3
뺑반
이월
jk닌자걸스
극장판 헬로카봇: 옴파로스 섬의 비밀
우리가족: 라멘샵
시인 할매
알리타: 배틀 엔젤
툼 인베이더
레고 무비 2
사랑은 비가 갠 뒤처럼
아이스
일진3
챔피언스
콜드 워
기묘한 가족
신밧드와 마법 양탄자
증인
닥터 킬러 패밀리
도쿄의 밤하늘은 항상 가장 짙은 블루
드래곤볼 슈퍼: 브로리
러브라이브! 선샤인!! 더 스쿨 아이돌 무비 오버 더 레인보우
메리 포핀스 리턴즈
메리, 퀸 오브 스코틀랜드
명탐정 코난:전율의 악보
타임 워프 워
해피 데스데이 2 유
해피니스 로드
험악한 꿈
5 솔저스
치트 더 행맨
포 핸즈
모험왕 블링키
사바하
신데렐라:마법 반지의 비밀
콜드 체이싱
CCTV: 은밀한 시선
더 페이버릿: 여왕의 여자
마지막 8월 어느 기념비의 이야기
살인마 잭의 집
허비 행콕: 무한한 가능성
에이리언 어택 2019
엽문 외전
더 와이프
래미의 초특급 시간여행
빠삐용
슈퍼미니 2
어쩌다, 결혼
옷코는 초등학생 사장님!
자전차왕 엄복동
칠곡 가시나들
퍼미션
항거: 유관순 이야기
600결사대
국경의 왕
실화: 숨겨진 비밀
캡틴 마블 : 예고

In [161]:
scores.to_csv('~/MovieTrends/output/scores.dat', sep='|', index=False)
scores.shape

(629, 4)

In [ ]:
movie_review_final = pd.concat(movie_reviews_list)

In [ ]:
# movie_review_final.to_csv("~/MovieTrends/output/movie_review_final.csv")

---

### DEBUG

In [ ]:
mv2019[mv2019["Title"] == "공포의 묘지"]

In [ ]:
mv2019.iloc[40:45]

In [ ]:
movie_nm = '공포의 묘지: 망자의 저주'
review_url, num_pages = get_review_info(movie_nm)
print(movie_nm, " | ", num_pages, "pages | ", review_url)

In [ ]:
movie_review = pd.DataFrame()
for i in range(1, num_pages+1):
    #print(movie_nm + " | " + str(i) + "/" + str(num_pages))
    movie_review_subset = get_movie_review_subset(movie_nm, review_url + '&page=' + str(i))
    movie_review = pd.concat([movie_review, movie_review_subset])
movie_reviews = pd.concat([movie_reviews, movie_review])

In [ ]:
resp = requests.get(url)
html = BeautifulSoup(resp.content, 'html.parser')
score_result = html.find('div', {'class': 'score_result'})
lis = score_result.findAll('li')
lis_df = pd.DataFrame()
for li in lis:
    nickname = li.findAll('a')[0].find('span').getText() if li.findAll('a')[0].find('span') is not None else None
    created_at = datetime.strptime(li.find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
    review_text = li.find('p').getText().translate(str.maketrans({"\n": "", "\r": "", "\t": ""}))
    score = li.find('em').getText()
    btn_likes = li.find('div', {'class': 'btn_area'}).findAll('strong')
    like = btn_likes[0].getText()
    dislike = btn_likes[1].getText()
    watch_movie = li.find('span', {'class':'ico_viewer'})
    
    li_df = pd.DataFrame({"movie_nm": [movie_nm],
                          "nickname": [nickname],
                          "review": [review_text],
                          "score": [score],
                          "like": [like],
                          "dislike": [dislike],
                          "created at": [created_at],
                          "watch_movie": [watch_movie and True or False]})
    lis_df = pd.concat([lis_df, li_df])

In [ ]:
get_movie_review_subset(movie_nm, review_url + '&page=' + str(i))

In [ ]:
review_url + '&page=' + str(i)

In [ ]:
movie_reviews = pd.concat(movie_review_list)

In [ ]:
movie_reviews.to_csv("~/MovieTrends/output/review_all.csv")

---

### FOR TEST

In [ ]:
review_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136990&type=after&page=1"
resp = requests.get(review_url)
html = BeautifulSoup(resp.content, 'html.parser')

In [ ]:
score_result = html.find('div', {'class': 'score_result'})
lis = score_result.findAll('li')
# lis[0]

In [ ]:
review_text = lis[0].find('p').getText()
review_text

In [ ]:
score = lis[0].find('em').getText()
score

In [ ]:
like = lis[0].find('div', {'class': 'btn_area'}).findAll('span')[1].getText()
dislike = lis[0].find('div', {'class': 'btn_area'}).findAll('span')[3].getText()
like, dislike

In [ ]:
nickname = lis[0].findAll('a')[0].find('span').getText()
nickname

In [ ]:
movie_nm = input("Enter movie name: ")
search_url = "https://movie.naver.com/movie/search/result.nhn?query=" + movie_nm + "&section=all&ie=utf8"
search_resp = requests.get(search_url)
search_html = BeautifulSoup(search_resp.content, 'html.parser')
a_tag = search_html.find('ul', {'class': 'search_list_1'}).find('a')

In [ ]:
a_tag

In [ ]:
re_movie = re.compile('code=[0-9]{6}')
movie_code = re.sub('code=', '', re_movie.findall(str(a_tag))[0])
review_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + movie_code + "&type=after&page=1"
return(review_url)